In [1]:
import pandas as pd

In [2]:
lig = pd.read_parquet("layerintegratedgradients.parquet")

In [3]:
lig

,sent,pos,token,value,layer,rel_value,sample,sample_list,step_count,sample_size
0,0,0,[CLS],0.016663,layer 0,0.03,0,[UNK],5,1
1,0,1,this,0.062638,layer 0,0.11,0,[UNK],5,1
2,0,2,movie,0.028054,layer 0,0.05,0,[UNK],5,1
3,0,3,was,-0.020720,layer 0,0.04,0,[UNK],5,1
4,0,4,the,0.177752,layer 0,0.31,0,[UNK],5,1
...,...,...,...,...,...,...,...,...,...,...
56,2,15,were,0.000000,layer 6,0.00,2,[SEP],5,1
57,2,16,well,0.000000,layer 6,0.00,2,[SEP],5,1
58,2,17,developed,0.000000,layer 6,0.00,2,[SEP],5,1
59,2,18,.,0.000000,layer 6,0.00,2,[SEP],5,1


In [4]:
lig['type'] = "IntegratedGradients"

In [5]:
leg = pd.read_parquet("layerexpectedgradients.parquet")

In [6]:
leg['type'] = "ExpectedGradients"

In [7]:
data = pd.concat([lig, leg])

In [8]:
from IPython.display import HTML
import re

def html_for_sentences(df, groupings):
    if len(groupings) > 1:
        output = "<div>"
        group = groupings[0]
        for g, gdf in df.groupby(group):
            output += f"<div>{group}: {g}</div><div style='padding-left:2em;'>"
            output += html_for_sentences(gdf, groupings[1:])
            output += "</div>"
        output += "</div>"
    else:
        output = "<table>"
        for label, lvl in df.groupby(groupings[0]):
            output += f"<tr><th>{label}</th>"
            for i, row in lvl.iterrows():
                pct = int(row['rel_value'] * 100)
                g = 10 + pct
                b = 150 + pct
                if row['value'] >= 0:
                    r = g
                else:
                    r = b
                a = (row['rel_value'] / 2) + 0.3
                color = ",".join(list(map(str, [r,g,b,a])))
                size = (0.5 + (row['rel_value'] / 2) + 0.3) * 1.5

                word = row['token'].strip()
                word = re.sub(r'<', '&lt;', re.sub(r'>', '&gt;', word))
                output += f"<td><span style='font-size: {size}em; color: rgba({color});'>{word}</span></td>"
            output += "</tr>"
        output += "</table>"
    return output


In [19]:
display(data[['type', 'sample', 'sample_list']].drop_duplicates())

,type,sample,sample_list
0,IntegratedGradients,0,[UNK]
0,IntegratedGradients,1,[PAD]
0,IntegratedGradients,2,[SEP]
0,ExpectedGradients,0,repair; pops; frequencies; juliana; convoys; a...
0,ExpectedGradients,1,alleged; clinched; auckland; obsession; archit...
0,ExpectedGradients,2,##ɲ; ##lift; objected; sammy; documentaries; #...


In [17]:
display(HTML(html_for_sentences(data, ['layer', 'sent', 'type', 'sample'])))